<a href="https://colab.research.google.com/github/Ironsoldier353/freelancing/blob/main/recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.2 MB/s eta 0:00:00


In [4]:
from sentence_transformers import SentenceTransformer, util
import re

# Load pre-trained Sentence-BERT model for optional semantic matching
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
from sentence_transformers import SentenceTransformer, util
import re

# Load pre-trained Sentence-BERT model for semantic matching
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Example Freelancer's profile
freelancer_profile = {
    "name": "John Doe",
    "skills": ["Python", "Natural Language Processing", "Deep Learning", "Machine Learning", "TensorFlow", "Keras"],
}

# Example list of job postings
job_postings = [
    {"title": "Data Scientist", "skills_required": ["Python", "Machine Learning", "Deep Learning", "TensorFlow"], "budget": 1500},
    {"title": "Web Developer", "skills_required": ["JavaScript", "HTML", "CSS", "React"], "budget": 1000},
    {"title": "ML Engineer", "skills_required": ["Machine Learning", "TensorFlow", "Python", "Keras"], "budget": 1800},
    {"title": "AI Researcher", "skills_required": ["NLP", "Deep Learning", "TensorFlow", "Python"], "budget": 2000},
    {"title": "Front-end Developer", "skills_required": ["JavaScript", "React", "HTML", "CSS"], "budget": 1200},
]

# Preprocess skills: Lowercase and remove special characters
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

# Calculate skill match score using semantic similarity
def calculate_semantic_skill_match(freelancer_skills, job_skills, threshold=0.7):
    matched_skills = []
    total_similarity = 0
    count = 0

    for job_skill in job_skills:
        # Get semantic similarity between job skill and freelancer's skills
        embeddings_job = model.encode(job_skill)

        # Check similarity for each freelancer's skill
        for freelancer_skill in freelancer_skills:
            embeddings_freelancer = model.encode(freelancer_skill)
            similarity = util.pytorch_cos_sim(embeddings_job, embeddings_freelancer).item()

            if similarity >= threshold:  # Threshold for considering it a match
                matched_skills.append(job_skill)
                total_similarity += similarity
                count += 1
                break

    # Calculate match score as average semantic similarity (between 0 and 100)
    skill_match_score = (total_similarity / len(job_skills)) * 100 if count > 0 else 0
    return skill_match_score, matched_skills

# Sort the jobs by match score first and budget second
def recommend_jobs(freelancer_profile, job_postings, similarity_threshold=0.7):
    freelancer_skills = [preprocess(skill) for skill in freelancer_profile["skills"]]
    recommendations = []

    # Calculate semantic skill match score for each job
    for job in job_postings:
        job_skills = [preprocess(skill) for skill in job["skills_required"]]

        # Calculate semantic match score
        match_score, matched_skills = calculate_semantic_skill_match(freelancer_skills, job_skills, threshold=similarity_threshold)

        # Add match score and matched skills to the job posting
        recommendations.append({
            "title": job["title"],
            "budget": job["budget"],
            "match_score": match_score,
            "matched_skills": matched_skills
        })

    # Sort by match score (descending), then by budget (descending)
    recommendations.sort(key=lambda x: (x['match_score'], x['budget']), reverse=True)

    return recommendations

# Generate job recommendations
recommended_jobs = recommend_jobs(freelancer_profile, job_postings)

# Display the sorted job recommendations
for job in recommended_jobs:
    print(f"Job Title: {job['title']}")
    print(f"Match Score: {job['match_score']:.2f}%")
    print(f"Budget: ${job['budget']}")
    print(f"Matched Skills: {', '.join(job['matched_skills'])}")
    print("-" * 40)


Job Title: Data Scientist
Match Score: 92.78%
Budget: $1500
Matched Skills: python, machine learning, deep learning, tensorflow
----------------------------------------
Job Title: ML Engineer
Match Score: 92.78%
Budget: $1800
Matched Skills: machine learning, tensorflow, python, keras
----------------------------------------
Job Title: AI Researcher
Match Score: 75.00%
Budget: $2000
Matched Skills: deep learning, tensorflow, python
----------------------------------------
Job Title: Front-end Developer
Match Score: 0.00%
Budget: $1200
Matched Skills: 
----------------------------------------
Job Title: Web Developer
Match Score: 0.00%
Budget: $1000
Matched Skills: 
----------------------------------------


In [6]:
# Example list of freelancers' profiles
freelancers = [
    {"name": "John Doe", "skills": ["Python", "Natural Language Processing", "Deep Learning", "Machine Learning", "TensorFlow", "Keras", "React.js"]},
    {"name": "Jane Smith", "skills": ["JavaScript", "HTML", "CSS", "React", "Node.js"]},
    {"name": "Alice Johnson", "skills": ["Python", "Machine Learning", "Deep Learning", "Keras", "TensorFlow"]},
    {"name": "Bob Lee", "skills": ["React", "Angular", "HTML", "CSS", "JavaScript"]},
    {"name": "Sara Wright", "skills": ["Python", "NLP", "Deep Learning", "Pytorch", "TensorFlow"]},
]

# Example job posting
job_posting = {
    "title": "ML Engineer",
    "skills_required": ["Machine Learning", "TensorFlow", "Python", "Keras","React"],
    "budget": 1800
}

# Preprocess skills: Lowercase and remove special characters
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

# Calculate skill match score using semantic similarity
def calculate_semantic_skill_match(freelancer_skills, job_skills, similarity_threshold=0.7):
    matched_skills = []
    total_similarity = 0

    # Preprocess freelancer and job skills
    freelancer_skills = [preprocess(skill) for skill in freelancer_skills]
    job_skills = [preprocess(skill) for skill in job_skills]

    # Encode job skills once
    job_skill_embeddings = model.encode(job_skills)

    # Keep track of matched job skills to avoid duplicates
    matched_job_indices = set()

    # Compare each freelancer skill to each job-required skill using cosine similarity
    for freelancer_skill in freelancer_skills:
        freelancer_skill_embedding = model.encode(freelancer_skill)

        # Compare with all job skills and get maximum similarity score
        similarities = util.pytorch_cos_sim(freelancer_skill_embedding, job_skill_embeddings)[0]
        max_similarity, best_match_idx = similarities.max(dim=0)

        # If similarity is above the threshold and the job skill has not been matched yet
        if max_similarity.item() >= similarity_threshold and best_match_idx.item() not in matched_job_indices:
            matched_skills.append(job_skills[best_match_idx.item()])
            total_similarity += max_similarity.item()
            matched_job_indices.add(best_match_idx.item())  # Mark this job skill as matched

    # Calculate match score based on number of matched skills
    skill_match_score = (len(matched_skills) / len(job_skills)) * 100 if len(job_skills) > 0 else 0
    return skill_match_score, matched_skills

# Sort freelancers by match score for a specific job
def recommend_freelancers(job_posting, freelancers, similarity_threshold=0.7):
    job_skills = job_posting["skills_required"]
    recommendations = []

    # Calculate combined skill match score for each freelancer
    for freelancer in freelancers:
        freelancer_skills = freelancer["skills"]

        # Calculate skill match score based on semantic similarity
        match_score, matched_skills = calculate_semantic_skill_match(freelancer_skills, job_skills, similarity_threshold)

        # Add match score and matched skills to the freelancer profile
        recommendations.append({
            "name": freelancer["name"],
            "match_score": min(match_score, 100),  # Cap the match score at 100%
            "matched_skills": matched_skills
        })

    # Sort by match score (descending)
    recommendations.sort(key=lambda x: x['match_score'], reverse=True)

    return recommendations

# Generate freelancer recommendations for the given job
recommended_freelancers = recommend_freelancers(job_posting, freelancers)

# Display the sorted freelancer recommendations
print(f"Job Title: {job_posting['title']} | Budget: ${job_posting['budget']}")
print("=" * 50)
for freelancer in recommended_freelancers:
    print(f"Freelancer: {freelancer['name']}")
    print(f"Match Score: {freelancer['match_score']:.2f}%")
    print(f"Matched Skills: {', '.join(freelancer['matched_skills'])}")
    print("-" * 40)


Job Title: ML Engineer | Budget: $1800
Freelancer: John Doe
Match Score: 100.00%
Matched Skills: python, machine learning, tensorflow, keras, react
----------------------------------------
Freelancer: Alice Johnson
Match Score: 80.00%
Matched Skills: python, machine learning, keras, tensorflow
----------------------------------------
Freelancer: Sara Wright
Match Score: 60.00%
Matched Skills: python, machine learning, tensorflow
----------------------------------------
Freelancer: Jane Smith
Match Score: 20.00%
Matched Skills: react
----------------------------------------
Freelancer: Bob Lee
Match Score: 20.00%
Matched Skills: react
----------------------------------------
